### Change the path according to your google drive directory

In [2]:
'''Parametres:
    img_scale: percentage resizing of image
    draw_lines: set this to False to only show Bounding boxes
    iou_threshold: a yolo parameter. you can play with it to see how it affects detections
    score_threshold: Since every detection has a score associated with it (between 0 and 1), this parameter sets the minimum score
    for a detection to be counted
'''
img_scale = 0.4
draw_lines = True
distance_threshold = 1.83 # Metres
iou_threshold = 0.1
score_threshold = 0.4

In [3]:
# Importing dependencies
import tensorflow as tf
#import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
from scipy.spatial import distance
import itertools

In [4]:
# Please make sure that it is 2.4.0 
print(tf.__version__)

2.4.0


In [5]:
# YOLOv4 Imports (ignore the v3 name in folder structure)
from yolov3.utils import Load_Yolo_model, image_preprocess, postprocess_boxes, nms, draw_detections, read_class_names
from yolov3.configs import *
import calibrate_frame

In [6]:
# Some helper functions
def compute_midpoint(p1, p2):
    '''Helper function to compute midpoint of two points
       Where each point is a tuple of integers
       Returns:
            list(p1[0]+p2[0]/2, p1[1]+p2[1]/2)
     '''
    mpX = (p1[0]+p2[0])/2
    mpY = (p1[1]+p2[1])/2
    return np.array([mpX, mpY], dtype=np.int16)

def compute_centroid(box):
    '''Helper function to compute centroid of box.
     Arguments:
            box: 1x4 np array of form:
                 box[0]: x1, box[1]: y1
                 box[2]: x2, box[3]: y2
     Returns: 
           nparray(centroidX, centroidY)
  '''
    return compute_midpoint((box[0], box[1]),(box[2], box[3]))

def convert_xc(box):
    '''Helper function to convert bbox to the x-coord,centroid form:
     [x1, x2, centre] This will be used to easily access x coords in
     ppm calculation function.
     Arguments:
              box: 1x4 np array of form:
                   box[0]: x1, box[1]: y1
                   box[2]: x2, box[3]: y2
     Returns:
            nparray([x1, x2, centre])
  '''
    centre = compute_centroid(box)
    return np.array([box[0], box[2], centre], dtype=object)

In [7]:
# Process input function
def get_bboxes(Yolo, frame, input_size, score_threshold, iou_threshold, limits):
    '''Process the input video and return bboxes in two point and w, h, c form
    frame: nd array of shape (height, width, channels)
  Returns:
     bboxes: list of bouding boxes in two point form
     bboxes_centroidForm: list of bboxes in w, h, c form
  '''
    try:
    # Change colour space
        original_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        original_frame = cv2.cvtColor(original_frame, cv2.COLOR_BGR2RGB)
    except:
        print('Exception while converting colour space. Returning')
        return
    
    # Preprocess as per YOLO's requirements
    image_data = image_preprocess(np.copy(original_frame),
                                [input_size, input_size])
    image_data = image_data[np.newaxis,...].astype(np.float32)

    # Get predictions
    pred_bbox = Yolo.predict(image_data)

    # Process the pred_bbox
    pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for x in pred_bbox]
    pred_bbox = tf.concat(pred_bbox, axis=0)
    bboxes = postprocess_boxes(pred_bbox, original_frame, input_size,
                                 score_threshold)
    bboxes = nms(bboxes, iou_threshold, method='nms', limits=limits)
    
    scores = list(map(lambda x: x[4], bboxes))
    # Get x Coord, centre form of bboxes
    bboxes_centroidForm = list(map(convert_xc, bboxes))
  
    return bboxes, bboxes_centroidForm, scores

def get_ppm(bboxes_xc, width, height, matrix):
    '''get_ppm function returns the pixels per metre for the reference object.
     Arguments:
              bboxes_xc: bounding boxes list in width,height,centroid form
     Returns:
              ppm: pixels per metre
              idx_middle: Index of middle person in bboxes
  '''
    # The average width of a person in metres
    approx_width_metre = 0.40
    # Centre point of input image
    centre = (int(width/2), int(height/2))
    # Fetch centroids
    centroids = list(map(lambda x: tuple(x[2]), bboxes_xc))
    distances = list(map(lambda x: distance.euclidean(centre, x), centroids))
    idx_middle = distances.index(min(distances))
    
    x1_middle, x2_middle = int(bboxes_xc[idx_middle][0]), int(bboxes_xc[idx_middle][1])
    x_coords = [[[x1_middle, x2_middle]]]
    print("XCOORDS OF MIDDLE GUY IN OG FRAME: {}".format(x_coords))
    x_coordsTransformed = cv2.perspectiveTransform(np.float32(x_coords), matrix) # This returns a (1,1,2) np array
    print("TRANSFORMED XCOORDS: {}".format(x_coordsTransformed))
    #width_middle = x_coordsTransformed[0,0,1] - x_coordsTransformed[0,0,0]
    width_middle = x2_middle - x1_middle
    ppm = width_middle/approx_width_metre
    print("WIDTH OF MIDDLE GUY IN ORIGINAL FRAME: {}".format(x2_middle-x1_middle))
    #print("WIDTH OF MIDDLE GUY IN TRANSFORMED FRAME: {}".format(width_middle))
    print("Pixels per Metre using approx width {}: {}".format(approx_width_metre, ppm))
    return ppm, idx_middle


def compute_distances(bboxes_xc, ppm, matrix):
    '''To calculate distance between all detected persons without repetitions
     Arguments:
              bboxes_xc: bounding boxes of form (x1, x2, centroid)
     Returns:
              distances = list(all distance)
              combinations
              centroids
  '''
    distances = []
    # Returns a list of form [[cx1, cy1], [cx2, cy2],...,[cxn, cyn]]
    centroids = list(map(lambda x: x[2], bboxes_xc))
    # Transforming centroids
    centroids_transformed = np.squeeze(cv2.perspectiveTransform(np.float32([centroids]), matrix))
    centroids_transformed = list(map(lambda x: tuple(x), centroids_transformed))
    # Combinations
    list_combinations = list(itertools.combinations(centroids_transformed, 2))
  
    for i, pair in enumerate(list_combinations):
        distances.append(distance.euclidean(pair[0], pair[1])/ppm)
    return distances, list(itertools.combinations(list(range(len(centroids))), 2)), list(map(lambda x: tuple(x[2]), bboxes_xc)), centroids_transformed


In [8]:
def process_video(input_video, output_path, fps):
    '''Process input function to perform detectios, overlays
       output_path1: Main video
       output_path2: Birds eye view
  '''
    # Set some detection parameters
    input_size = 416
    # Number of frames 
    N = input_video.shape[0]
    
    # First frame for calibration
    frame = input_video[0, :,:,:]
    
    # Get transformation matrix 
    matrix, inv_matrix, list_points = calibrate_frame.calibrate(frame)
    
    # max_limits = [[minx, miny], [maxx, maxy]]
    limits =  [[min(list(map(lambda x: x[0], list_points))), min(list(map(lambda x: x[1], list_points)))],
                   [max(list(map(lambda x: x[0], list_points))), max(list(map(lambda x: x[1], list_points)))]]
    
    # Get frame dimensions and set codec info
    width = input_video.shape[2]
    height = input_video.shape[1]
    codec = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path+'Main.mp4', codec, fps,
                        (width,height))
    out_bird = cv2.VideoWriter(output_path+'Birdseye.mp4', codec, fps, (width,height))
    yolo = Load_Yolo_model()
    ppm = None
    
    # Start reading frames
    for i in range(N):
        # Birdseye
        birds_eye = np.zeros((height, width, 3), dtype=np.uint8)
        # OG Frame
        frame = input_video[i, :,:,:]
        # Get bounding boxes from yolov4
        bboxes, bboxes_xc, scores = get_bboxes(yolo, frame, input_size, score_threshold,
                                        iou_threshold, limits)
        
        if len(bboxes) == 0:
            print("No detections in frame {}".format(i))
            continue
        else:
            print("frame {}, detections {}".format(i, len(bboxes)))
            
        # Calculate ppm and distances only if draw_lines is set
        if draw_lines:
            if ppm == None:
                ppm, idx_middle = get_ppm(bboxes_xc, frame.shape[1], frame.shape[0], matrix)
            distances, combinations, centroids, centroids_transformed = compute_distances(bboxes_xc, ppm, matrix)
            frame, birds_eye = draw_detections(frame, bboxes, birds_eye, combinations, distances, centroids, centroids_transformed,
                                              distance_threshold)
        else:
             frame = draw_bbox(frame, bboxes)   
        
    
        out.write(frame)
        out_bird.write(birds_eye)
        cv2.imshow('OG Frame', frame)
        cv2.imshow('Birds Eye', birds_eye)
        cv2.waitKey(1000)
        
    out.release()
    cv2.destroyAllWindows()


In [9]:
def process_image(input_path, output_path):
    frame = cv2.imread(input_path)
    frame = cv2.resize(frame, (int(frame.shape[1]*img_scale), int(frame.shape[0]*img_scale)))
    
    # Birdseye
    birds_eye = np.zeros((frame.shape[0], frame.shape[1], 3), dtype=np.uint8)
    # Set some detection parameters
    input_size = 416
    yolo = Load_Yolo_model()
    
    # Get transformation matrix 
    matrix, inv_matrix, list_points = calibrate_frame.calibrate(frame)
    # max_limits = [[minx, miny], [maxx, maxy]]
    limits =  [[min(list(map(lambda x: x[0], list_points))), min(list(map(lambda x: x[1], list_points)))],
                   [max(list(map(lambda x: x[0], list_points))), max(list(map(lambda x: x[1], list_points)))]]
  
    # Get bounding boxes from yolov4
    bboxes, bboxes_xc, scores = get_bboxes(yolo, frame, input_size, score_threshold,
                                     iou_threshold, limits)
    if len(bboxes) == 0:
        print("No detections in image returning")
        return
    else:
        print("# detections {}".format(len(bboxes)))
    
    # Calculate ppm and distances only if required
    if draw_lines:
        ppm, idx_middle = get_ppm(bboxes_xc, frame.shape[1], frame.shape[0], matrix)
        distances, combinations, centroids, centroids_transformed = compute_distances(bboxes_xc, ppm, matrix)
        frame, birds_eye = draw_detections(frame, bboxes, birds_eye, combinations, distances, centroids, centroids_transformed,
                                              distance_threshold)
    else:
        frame = draw_bbox(frame, bboxes)
        
    # Save images
    cv2.imwrite(output_path+'Main.jpg', frame)
    cv2.imwrite(output_path+'birdseye.jpg', birds_eye)
    
    # Display both images
    cv2.imshow('OG Frame', frame)
    cv2.imshow('Birds Eye', birds_eye)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

### Change the filepath in according to the input image/video

In [11]:
# For image
process_image('c:/users/moham/desktop/Upwork/Social distancing/Samples/testim5.png','c:/users/moham/desktop/Upwork/Social distancing/Outputs/testim5Out')

Loading Darknet_weights from: model_data/yolov4.weights
Calibration started
Width,Height of frame: (780,520)
Left Mouse click at ( 21 , 95 )
Left Mouse click at ( 496 , 112 )
Left Mouse click at ( 13 , 510 )
Left Mouse click at ( 554 , 503 )
Image to birds eye matrix: [[ 1.59855506e+00  3.08155193e-02 -3.64971307e+01]
 [-5.30414761e-02  1.48204124e+00 -1.39680047e+02]
 [-1.34177752e-04  3.63418137e-04  1.00000000e+00]]
Birds eye to image matrix: [[ 6.46545466e-01 -1.85928801e-02  2.10000000e+01]
 [ 3.02786701e-02  6.72214230e-01  9.50000000e+01]
 [ 7.57481995e-05 -2.46789594e-04  1.00000000e+00]]
# detections 2
XCOORDS OF MIDDLE GUY IN OG FRAME: [[[207, 261]]]
TRANSFORMED XCOORDS: [[[283.43457 221.30835]]]
WIDTH OF MIDDLE GUY IN ORIGINAL FRAME: 54
Pixels per Metre using approx width 0.4: 135.0


In [12]:
# For video
cap = cv2.VideoCapture('c:/users/moham/desktop/upwork/social distancing/Samples/PETS2009.avi')
frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = int(cap.get(cv2.CAP_PROP_FPS))
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frameCount = 30
buf = np.empty((frameCount, int(frameHeight*img_scale), int(frameWidth*img_scale), 3), np.dtype('uint8'))
fc = 0
ret = True

while (fc < frameCount and ret):
    ret, frame = cap.read()
    frame = cv2.resize(frame, (int(frameWidth*img_scale), int(frameHeight*img_scale)))
    buf[fc] = frame
    fc += 1

cap.release()

In [13]:
print(buf.shape)

(30, 230, 307, 3)


### Set the output path in the second argument of process_input. Make sure that it ends in .mp4

In [ ]:
process_video(buf[:, :,:,:], 'c:/users/moham/desktop/upwork/social distancing/outputs/PETS2009out', fps)

Calibration started
Width,Height of frame: (307,230)
Left Mouse click at ( 94 , 46 )
Left Mouse click at ( 297 , 88 )
Left Mouse click at ( 46 , 194 )
Left Mouse click at ( 252 , 215 )
Image to birds eye matrix: [[ 1.13622211e+00  3.68504467e-01 -1.23756084e+02]
 [-2.74960822e-01  1.32897731e+00 -3.52866389e+01]
 [-5.74377586e-04 -3.14574722e-04  1.00000000e+00]]
Birds eye to image matrix: [[ 8.17877613e-01 -2.04534365e-01  9.40000000e+01]
 [ 1.83219619e-01  6.61028035e-01  4.60000000e+01]
 [ 5.27406830e-04  9.04627555e-05  1.00000000e+00]]
Loading Darknet_weights from: model_data/yolov4.weights
frame 0, detections 3
XCOORDS OF MIDDLE GUY IN OG FRAME: [[[102, 122]]]
TRANSFORMED XCOORDS: [[[ 41.07936 109.41165]]]
WIDTH OF MIDDLE GUY IN ORIGINAL FRAME: 20
Pixels per Metre using approx width 0.4: 50.0
frame 1, detections 3
frame 2, detections 4
frame 3, detections 4
frame 4, detections 3
frame 5, detections 3
frame 6, detections 3
frame 7, detections 3
frame 8, detections 3
frame 9, detec